### Connection & Imports

In [1]:
from sqlalchemy.orm import Session
import sys
sys.path.append('..')
from app import models
from app.database import engine,SessionLocal
from pprint import pprint
import pandas as pd
from fastapi.encoders import jsonable_encoder
import random

### Drop all

In [2]:
models.Base.metadata.drop_all(bind=engine)

### Create all

In [3]:
models.Base.metadata.create_all(bind=engine)

### Table Inspection

In [4]:
from sqlalchemy.engine import reflection
from IPython.display import HTML
insp = reflection.Inspector.from_engine(engine)

for table_name in insp.get_table_names():
    display(HTML(f"<h3>Table: {table_name}</h3>"))
    display(HTML(pd.DataFrame(insp.get_columns(table_name)).to_html(index=False)))

name,type,nullable,default,autoincrement,primary_key
abteilung_id,INTEGER,False,None,auto,1
name,VARCHAR(50),True,None,auto,0


name,type,nullable,default,autoincrement,primary_key
user_id_aktiv,INTEGER,False,None,auto,1
user_id_passiv,INTEGER,False,None,auto,2


name,type,nullable,default,autoincrement,primary_key
dummy_antwort_id,INTEGER,False,None,auto,1
frage_id,INTEGER,True,None,auto,0
antwort,VARCHAR(200),True,None,auto,0


name,type,nullable,default,autoincrement,primary_key
erfolg_id,INTEGER,False,None,auto,1
name,VARCHAR(50),True,None,auto,0
bildpfad,VARCHAR(200),True,None,auto,0


name,type,nullable,default,autoincrement,primary_key
user_id,INTEGER,False,None,auto,1
erfolg_id,INTEGER,False,None,auto,2


name,type,nullable,default,autoincrement,primary_key
frage_id,INTEGER,False,None,auto,1
frage,VARCHAR(200),True,None,auto,0


name,type,nullable,default,autoincrement,primary_key
steckbrief_id,INTEGER,False,None,auto,1
freigegeben,BOOLEAN,True,'1',auto,0


name,type,nullable,default,autoincrement,primary_key
steckbrief_id,INTEGER,False,None,auto,1
frage_id,INTEGER,False,None,auto,2
antwort,VARCHAR(200),True,None,auto,0


name,type,nullable,default,autoincrement,primary_key
user_id,INTEGER,False,None,auto,1
abteilung_id,INTEGER,True,None,auto,0
steckbrief_id,INTEGER,True,None,auto,0
beruf,VARCHAR(50),True,None,auto,0
vorname,VARCHAR(50),True,None,auto,0
nachname,VARCHAR(50),True,None,auto,0
geburtsdatum,DATE,True,None,auto,0
wohnort,VARCHAR(50),True,None,auto,0


In [5]:
df=pd.read_csv("../db/testdata/abteilung.csv",sep=";")
df.to_sql("abteilung",engine, if_exists='append',index=False)

In [6]:
df=pd.read_csv("../db/testdata/users.csv",sep=";")
df.to_sql("user",engine, if_exists='append',index=False)

In [7]:
df=pd.read_csv("../db/testdata/steckbriefe.csv",sep=";")
df.to_sql("steckbrief",engine, if_exists='append',index=False)

In [8]:
df=pd.read_csv("../db/testdata/steckbrief_frage.csv",sep=";")
df.to_sql("steckbrief_frage",engine, if_exists='append',index=False)

In [9]:
df=pd.read_csv("../db/testdata/frage.csv",sep=";")
df.to_sql("frage",engine, if_exists='append',index=False)

In [10]:
df=pd.read_csv("../db/testdata/dummy_antwort.csv",sep=";")
df.to_sql("dummy_antwort",engine, if_exists='append',index=False)

In [11]:
df=pd.read_csv("../db/testdata/collection.csv",sep=";")
df.to_sql("collection",engine, if_exists='append',index=False)

In [14]:
df=pd.read_csv("../db/testdata/erfolg.csv",sep=";")
df.to_sql("erfolg",engine, if_exists='append',index=False)

In [52]:
df=pd.read_csv("../db/testdata/erfolg_user.csv",sep=";")
df.to_sql("erfolg_user",engine, if_exists='append',index=False)

In [12]:
break()

SyntaxError: invalid syntax (<ipython-input-12-4e16a628866f>, line 1)

### Examples

In [16]:
db = SessionLocal()

In [17]:
user_id=2

In [41]:
db.query(models.Collection).filter(models.Collection.user_id_aktiv==user_id).count()

4

In [65]:
from sqlalchemy import case


collection = db.query(models.Erfolg_User).filter(models.Erfolg_User.user_id==user_id).subquery()

check_collected = case([
    (collection.c.user_id != None, 1)],
        else_=0
)


result=db.query(models.Erfolg.erfolg_id,models.Erfolg.name,models.Erfolg.bildpfad,check_collected.label('gesammelt')).outerjoin(collection,models.Erfolg.erfolg_id==collection.c.erfolg_id).all()
[dict(zip(['id','name','bildpfad','gesammelt'],r)) for r in result]

[{'id': 1, 'name': '1_User', 'bildpfad': 'erfolg1', 'gesammelt': 1},
 {'id': 2, 'name': '5_User', 'bildpfad': 'erfolg2', 'gesammelt': 0},
 {'id': 3, 'name': '10_User', 'bildpfad': 'erfolg3', 'gesammelt': 0}]

In [ ]:
from sqlalchemy import case

collection = db.query(models.Collection).filter(models.Collection.user_id_aktiv==user_id).subquery()
abteilung=models.Abteilung
user=models.User
users=db.query(user.user_id,
     abteilung.name.label('abteilung'),
     user.steckbrief_id,
     user.beruf,
     user.vorname,
     user.nachname,
     user.geburtsdatum,
     user.wohnort).join(abteilung).subquery()


check_collected = case([
    (collection.c.user_id_aktiv != None, 1)],
        else_=0
)

db.query(*users.c,check_collected.label('gesammelt')).outerjoin(collection,users.c.user_id==collection.c.user_id_passiv).filter(users.c.user_id!=user_id).all()

In [ ]:
pd.read_sql(SessionLocal().query(models.User).statement,SessionLocal().bind).T.to_dict().values()

In [ ]:
from sqlalchemy import case

my_case = case([
        (collection.user_id_aktiv != None, 1)],
         else_=0
)

In [ ]:
collection = SessionLocal().query(models.Collection).filter(models.Collection.user_id_aktiv==2).subquery()


my_case = case([
        (collection.c.user_id_aktiv != None, 1)],
         else_=0
)

pd.read_sql(SessionLocal().query(models.User,my_case.label('gesammelt')).outerjoin(collection,models.User.user_id==collection.c.user_id_passiv).filter(models.User.user_id!=2).statement, SessionLocal().bind)

In [ ]:
pd.read_sql(SessionLocal().query(models.Collection.user_id_aktiv,models.Collection.user_id_passiv).filter(models.Collection.user_id_aktiv==2).statement, SessionLocal().bind)

In [ ]:
collection = SessionLocal().query(models.Collection).filter(models.Collection.user_id_aktiv==2).subquery()
    
check_collected = case([
    (collection.c.user_id_aktiv != None, 1)],
        else_=0
)

print(SessionLocal().query(models.User,check_collected.label('gesammelt')).outerjoin(collection,models.User.user_id==collection.c.user_id_passiv).filter(models.User.user_id!=2))

In [ ]:
for row in SessionLocal().query(models.User,my_case.label('gesammelt')).outerjoin(collection,models.User.user_id==collection.c.user_id_passiv).filter(models.User.user_id!=2).all():
    print(row[0].vorname)